In [ ]:
import csv
from bs4 import BeautifulSoup
# Firefox ve Crome için
from selenium import webdriver
# MsEdge için
from msedge.selenium_tools import Edge, EdgeOptions

In [ ]:
#Firefox ve Crome için
#driver = webdriver.Firefox()
#driver = webdriver.Chrome()

#Edge için
options = EdgeOptions()
options.use_chromium = True
driver = Edge(options = options)

In [ ]:
url = 'https://www.amazon.com'
driver.get(url)

In [ ]:
# Bir arama yapıldığında (kindle) örnek link şu şekildedir:
# https://www.amazon.com/s?k=kindle&sprefix=kin%2Caps%2C243&ref=nb_sb_ss_ts-doa-p_1_3
#ortadaki kindle yazısına ulaşmak için:

def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&sprefix=kin%2Caps%2C243&ref=nb_sb_ss_ts-doa-p_1_3'
    #aramada birden fazla kelime olursa aralarına '+' işareti atmak için;
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

In [ ]:
#örnek link göründü
url = get_url('kindle')
print(url)

In [ ]:
driver.get_url

Veri Toplama

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [ ]:
# istenen girdi ile ilgili tüm değerleri getirir
results = soup.find_all('div', {'data-component-type': 's-search-result'})
len(results)

Url'yi çekmek

In [ ]:
item = results[0]

In [ ]:
# amazon sitesini incelersek ürün başlıklarını h2 tag'i içinde göreceğiz
atag = item.h2.a

In [ ]:
description = atag.text.strip()

In [ ]:
url = 'https://www.amazon.com' + atag.get('href')

Fiyatları Çekmek

In [ ]:
#span'ın içindeki a-price tag'i ana tag'imizdir. Onun da içindeki a-offscreen tag'inde ise fiyatlar yazmaktadır.
price_parent = item.find('span', 'a-price')

In [ ]:
price_parent.find('span', 'a-offscreen')

In [ ]:
price = price_parent.find('span', 'a-offscreen').text

Puanları Çekmek

In [ ]:
rating = item.i.text

In [ ]:
#Puanları ve toplam oy sayısını gösterir
review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text

Hepsini tek bir yerde göstermek için

In [ ]:
def extract_record(item):
    #açıklama ve url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    #fiyat
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
    
    #puan
    rating = item.i.text
    review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    
    
    result = (description, price, rating, review_count, url)
    return result

In [ ]:
records = []
results = soup.find_all("div{'data-component-type': 's-search-result}")

for item in results:
    records.append(extract_records(item))

Hata Ayıklama

In [ ]:
#Bazı ürünlerin fiyatları veya puanları olmadığı için hata verecektir. Bunu düzeltmek için:

In [ ]:
def extract_record(item):
    #açıklama ve url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    #fiyat
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return    
    #puan
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        rating = ' '
        review_count = ' '
    
    result = (description, price, rating, review_count, url)
    return result

In [ ]:
records = []
results = soup.find_all("div{'data-component-type': 's-search-result}")

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

In [ ]:
#örnek çıktı görmek için
record[0]

In [ ]:
#örneğin bütün fiyatları görmek istersek
for row in records:
    print(row[1])

Sonraki sayfalara erişmek

In [ ]:
#Eğer kindle yazıp 2. sayfaya tıklarsam link şu şekilde olur:
#https://www.amazon.com.tr/s?k=kindle&page=2&__mk_tr_TR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=ITUXR3K0FLF5&qid=1660681078&sprefix=kindle%2Caps%2C224&ref=sr_pg_2
#linkin içinde kindle'dan sonra &'ler içinde page=2 yazmaktadır. Bu kısıma ulaşmak gerekmektedir.


def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&sprefix=kin%2Caps%2C243&ref=nb_sb_ss_ts-doa-p_1_3'
    #aramada birden fazla kelime olursa aralarına '+' işareti atmak için;
    search_term = search_term.replace(' ', '+')
    url = template.format(search_term)
    
    #url'ye placeholder ekleme
    url += '&page{}'
    
    return url

Hepsini Birleştirmek -- Sonuç

Diğer kısımların toplamı olduğu için sadece bu kısım çalıştırılsa yeterli olacaktır

In [ ]:
import csv
from bs4 import BeautifulSoup
# Firefox ve Crome için
from selenium import webdriver
# MsEdge için
from msedge.selenium_tools import Edge, EdgeOptions

def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&sprefix=kin%2Caps%2C243&ref=nb_sb_ss_ts-doa-p_1_3'
    #aramada birden fazla kelime olursa aralarına '+' işareti atmak için;
    search_term = search_term.replace(' ', '+')
    url = template.format(search_term)
    
    #url'ye placeholder ekleme
    url += '&page{}'
    
    return url

def extract_record(item):
    #açıklama ve url
    atag = item.h2.a
    description = atag.text.strip()
    url = 'https://www.amazon.com' + atag.get('href')
    
    #fiyat
    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return    
    #puan
    try:
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        rating = ' '
        review_count = ' '
    
    result = (description, price, rating, review_count, url)
    return result
# ana fonksiyon oluşturulup her şey bunun içinde gösterilecektir.
def main(search_term):
    
    options = EdgeOptions()
    options.use_chromium = True
    driver = Edge(options = options)
    
    record = []
    url = get_url(search_term)
    records = []
    
    for page in range (1, 8): # toplam 7 sayfaya da erişmek istediğim için 1 ile 8 arasını kullandım.
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    driver.close()
    
    #csv'ye kaydetmek
    with open('amazon_ws.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description', 'Price', 'Rating', 'Review_Count', 'URL'])
        writer.writerows(records)

main('kindle')